<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/ShopRunner/collie/blob/main/tutorials/05_hybrid_model.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/ShopRunner/collie/blob/main/tutorials/05_hybrid_model.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://raw.githubusercontent.com/ShopRunner/collie/main/tutorials/05_hybrid_model.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

In [1]:
# for Collab notebooks, we will start by installing the ``collie`` library
!pip install collie --quiet

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

%env DATA_PATH data/

env: DATA_PATH=data/


In [3]:
import os

import numpy as np
import pandas as pd
from pytorch_lightning.utilities.seed import seed_everything
from IPython.display import HTML
import joblib
import torch

from collie.metrics import mapk, mrr, auc, evaluate_in_batches
from collie.model import CollieTrainer, HybridPretrainedModel, MatrixFactorizationModel
from collie.movielens import get_movielens_metadata, get_recommendation_visualizations, get_user_metadata

## Load Data From ``01_prepare_data`` Notebook 
If you're running this locally on Jupyter, you should be able to run the next cell quickly without a problem! If you are running this on Colab, you'll need to regenerate the data by running the cell below that, which should only take a few extra seconds to complete. 

In [4]:
try:
    # let's grab the ``Interactions`` objects we saved in the last notebook
    train_interactions = joblib.load(os.path.join(os.environ.get('DATA_PATH', 'data/'),
                                                  'train_interactions.pkl'))
    val_interactions = joblib.load(os.path.join(os.environ.get('DATA_PATH', 'data/'),
                                                'val_interactions.pkl'))
except FileNotFoundError:
    # we're running this notebook on Colab where results from the first notebook are not saved
    # regenerate this data below
    from collie.cross_validation import stratified_split
    from collie.interactions import Interactions
    from collie.movielens import read_movielens_df
    from collie.utils import convert_to_implicit, remove_users_with_fewer_than_n_interactions


    df = read_movielens_df(decrement_ids=True)
    implicit_df = convert_to_implicit(df, min_rating_to_keep=4)
    implicit_df = remove_users_with_fewer_than_n_interactions(implicit_df, min_num_of_interactions=3)

    interactions = Interactions(
        users=implicit_df['user_id'],
        items=implicit_df['item_id'],
        ratings=implicit_df['rating'],
        allow_missing_ids=True,
    )

    train_interactions, val_interactions = stratified_split(interactions, test_p=0.1, seed=42)


print('Train:', train_interactions)
print('Val:  ', val_interactions)

Checking for and removing duplicate user, item ID pairs...
Checking ``num_negative_samples`` is valid...
Maximum number of items a user has interacted with: 378
Generating positive items set...
Generating positive items set...
Generating positive items set...
Train: Interactions object with 49426 interactions between 943 users and 1674 items, returning 10 negative samples per interaction.
Val:   Interactions object with 5949 interactions between 943 users and 1674 items, returning 10 negative samples per interaction.


# Hybrid Collie Model Using a Pre-Trained ``MatrixFactorizationModel``
In this notebook, we will use this same metadata and incorporate it directly into the model architecture with a hybrid Collie model. 

## Read in Data

In [5]:
# read in the same metadata used in notebooks ``03`` and ``04``
metadata_df = get_movielens_metadata()


metadata_df.head()

,genre_action,genre_adventure,genre_animation,genre_children,genre_comedy,genre_crime,genre_documentary,genre_drama,genre_fantasy,genre_film_noir,...,genre_unknown,decade_unknown,decade_20,decade_30,decade_40,decade_50,decade_60,decade_70,decade_80,decade_90
0,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [6]:
# read in the user metadata
user_metadata_df = get_user_metadata()


user_metadata_df.head()

,age,gender,occupation_administrator,occupation_artist,occupation_doctor,occupation_educator,occupation_engineer,occupation_entertainment,occupation_executive,occupation_healthcare,...,occupation_marketing,occupation_none,occupation_other,occupation_programmer,occupation_retired,occupation_salesman,occupation_scientist,occupation_student,occupation_technician,occupation_writer
0,24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,53,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,33,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [7]:
# and, as always, set our random seed
seed_everything(22)

Global seed set to 22


22

## Train a ``MatrixFactorizationModel`` 

The first step towards training a Collie Hybrid model is to train a regular ``MatrixFactorizationModel`` to generate rich user and item embeddings. We'll use these embeddings in a ``HybridPretrainedModel`` a bit later. 

In [8]:
model = MatrixFactorizationModel(
    train=train_interactions,
    val=val_interactions,
    embedding_dim=30,
    lr=1e-2,
)

In [9]:
trainer = CollieTrainer(model=model, max_epochs=10, deterministic=True)

trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name            | Type            | Params
----------------------------------------------------
0 | user_biases     | ZeroEmbedding   | 943   
1 | item_biases     | ZeroEmbedding   | 1.7 K 
2 | user_embeddings | ScaledEmbedding | 28.3 K
3 | item_embeddings | ScaledEmbedding | 50.2 K
4 | dropout         | Dropout         | 0     
----------------------------------------------------
81.1 K    Trainable params
0         Non-trainable params
81.1 K    Total params
0.325     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 22
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:413: UserWarning: The number of training samples (49) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Epoch     7: reducing learning rate of group 0 to 1.0000e-05.
Epoch     7: reducing learning rate of group 0 to 1.0000e-05.


Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Epoch     9: reducing learning rate of group 0 to 1.0000e-06.
Epoch     9: reducing learning rate of group 0 to 1.0000e-06.


Validating: 0it [00:00, ?it/s]

In [10]:
mapk_score, mrr_score, auc_score = evaluate_in_batches([mapk, mrr, auc], val_interactions, model)

print(f'Standard MAP@10 Score: {mapk_score}')
print(f'Standard MRR Score:    {mrr_score}')
print(f'Standard AUC Score:    {auc_score}')

  0%|          | 0/48 [00:00<?, ?it/s]

Standard MAP@10 Score: 0.03774576080333839
Standard MRR Score:    0.13791782394715935
Standard AUC Score:    0.9061467564030058


## Train a ``HybridPretrainedModel`` 

With our trained ``model`` above, we can now use these embeddings and additional side data directly in a hybrid model. The architecture essentially takes our user embedding, item embedding, item metadata, and user metadata for each user-item interaction, concatenates them, and sends it through a simple feedforward network to output a recommendation score. 

We can initially freeze the user and item embeddings from our previously-trained ``model``, train for a few epochs only optimizing our newly-added linear layers, and then train a model with everything unfrozen at a lower learning rate. We will show this process below. 

In [11]:
# we will apply a linear layer to the metadata with ``metadata_layers_dims`` and
# a linear layer to the combined embeddings and metadata data with ``combined_layers_dims``
hybrid_model = HybridPretrainedModel(
    train=train_interactions,
    val=val_interactions,
    item_metadata=metadata_df,
    user_metadata=user_metadata_df,
    trained_model=model,
    item_metadata_layers_dims=[8],
    user_metadata_layers_dims=[8],
    combined_layers_dims=[16],
    lr=1e-2,
    freeze_embeddings=True,
)

In [12]:
hybrid_trainer = CollieTrainer(model=hybrid_model, max_epochs=10, deterministic=True)

hybrid_trainer.fit(hybrid_model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name                  | Type                     | Params
-------------------------------------------------------------------
0 | _trained_model        | MatrixFactorizationModel | 81.1 K
1 | embeddings            | Sequential               | 78.5 K
2 | biases                | Sequential               | 2.6 K 
3 | dropout               | Dropout                  | 0     
4 | item_metadata_layer_0 | Linear                   | 232   
5 | user_metadata_layer_0 | Linear                   | 192   
6 | combined_layer_0      | Linear                   | 1.2 K 
7 | combined_layer_1      | Linear                   | 17    
-------------------------------------------------------------------
85.4 K    Trainable params
78.5 K    Non-trainable params
163 K     Total params
0.656     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 22


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Epoch     7: reducing learning rate of group 0 to 1.0000e-05.


Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Epoch     9: reducing learning rate of group 0 to 1.0000e-06.


Validating: 0it [00:00, ?it/s]

In [13]:
mapk_score, mrr_score, auc_score = evaluate_in_batches([mapk, mrr, auc], val_interactions, hybrid_model)

print(f'Hybrid MAP@10 Score: {mapk_score}')
print(f'Hybrid MRR Score:    {mrr_score}')
print(f'Hybrid AUC Score:    {auc_score}')

  0%|          | 0/48 [00:00<?, ?it/s]

Hybrid MAP@10 Score: 0.04742431106869107
Hybrid MRR Score:    0.16543878608575038
Hybrid AUC Score:    0.9057771792583911


In [14]:
hybrid_model_unfrozen = HybridPretrainedModel(
    train=train_interactions,
    val=val_interactions,
    item_metadata=metadata_df,
    user_metadata=user_metadata_df,
    trained_model=model,
    item_metadata_layers_dims=[8],
    user_metadata_layers_dims=[8],
    combined_layers_dims=[16],
    lr=1e-4,
    freeze_embeddings=False,
)

hybrid_model.unfreeze_embeddings()
hybrid_model_unfrozen.load_from_hybrid_model(hybrid_model)

In [15]:
hybrid_trainer_unfrozen = CollieTrainer(model=hybrid_model_unfrozen, max_epochs=10, deterministic=True)

hybrid_trainer_unfrozen.fit(hybrid_model_unfrozen)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name                  | Type                     | Params
-------------------------------------------------------------------
0 | _trained_model        | MatrixFactorizationModel | 81.1 K
1 | embeddings            | Sequential               | 78.5 K
2 | biases                | Sequential               | 2.6 K 
3 | dropout               | Dropout                  | 0     
4 | item_metadata_layer_0 | Linear                   | 232   
5 | user_metadata_layer_0 | Linear                   | 192   
6 | combined_layer_0      | Linear                   | 1.2 K 
7 | combined_layer_1      | Linear                   | 17    
-------------------------------------------------------------------
85.4 K    Trainable params
78.5 K    Non-trainable params
163 K     Total params
0.656     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 22


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Epoch     8: reducing learning rate of group 0 to 1.0000e-05.


Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Epoch    10: reducing learning rate of group 0 to 1.0000e-06.


In [16]:
mapk_score, mrr_score, auc_score = evaluate_in_batches([mapk, mrr, auc],
                                                       val_interactions,
                                                       hybrid_model_unfrozen)

print(f'Hybrid Unfrozen MAP@10 Score: {mapk_score}')
print(f'Hybrid Unfrozen MRR Score:    {mrr_score}')
print(f'Hybrid Unfrozen AUC Score:    {auc_score}')

  0%|          | 0/48 [00:00<?, ?it/s]

Hybrid Unfrozen MAP@10 Score: 0.045106490775912465
Hybrid Unfrozen MRR Score:    0.16049974277267537
Hybrid Unfrozen AUC Score:    0.9084340184610614


Note here that while our ``MAP@10`` and ``MRR`` scores went down slightly from the frozen version of the model above, our ``AUC`` score increased. For implicit recommendation models, each evaluation metric is nuanced in what it represents for real world recommendations. 

You can read more about each evaluation metric by checking out the [Mean Average Precision at K (MAP@K)](https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Mean_average_precision), [Mean Reciprocal Rank](https://en.wikipedia.org/wiki/Mean_reciprocal_rank), and [Area Under the Curve (AUC)](https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve) Wikipedia pages. 

In [17]:
user_id = np.random.randint(0, train_interactions.num_users)

display(
    HTML(
        get_recommendation_visualizations(
            model=hybrid_model_unfrozen,
            user_id=user_id,
            filter_films=True,
            shuffle=True,
            detailed=True,
        )
    )
)

,Beauty and the Beast (1991),Dances with Wolves (1990),E.T. the Extra-Terrestrial (1982),Indiana Jones and the Last Crusade (1989),Alice in Wonderland (1951),How to Make an American Quilt (1995),Jerry Maguire (1996),Cinderella (1950),Toy Story (1995),Mary Poppins (1964)
Some loved films:,,,,,,,,,,
,Mr. Holland's Opus (1995),Star Wars (1977),Return of the Jedi (1983),"Empire Strikes Back, The (1980)",Independence Day (ID4) (1996),Mrs. Doubtfire (1993),Braveheart (1995),Ransom (1996),When Harry Met Sally... (1989),Star Trek: First Contact (1996)
Recommended films:,,,,,,,,,,


The metrics and results look great, and we should only see a larger difference compared to a standard model as our data becomes more nuanced and complex (such as with MovieLens 10M data). 

If we're happy with this model, we can go ahead and save it for later! 

## Save and Load a Hybrid Model 

In [18]:
# we can save the model with...
os.makedirs('models', exist_ok=True)
hybrid_model_unfrozen.save_model('models/hybrid_model_unfrozen')

In [19]:
# ... and if we wanted to load that model back in, we can do that easily...
hybrid_model_loaded_in = HybridPretrainedModel(load_model_path='models/hybrid_model_unfrozen')


hybrid_model_loaded_in

HybridPretrainedModel(
  (embeddings): Sequential(
    (0): ScaledEmbedding(943, 30)
    (1): ScaledEmbedding(1674, 30)
  )
  (biases): Sequential(
    (0): ZeroEmbedding(943, 1)
    (1): ZeroEmbedding(1674, 1)
  )
  (dropout): Dropout(p=0.0, inplace=False)
  (item_metadata_layer_0): Linear(in_features=28, out_features=8, bias=True)
  (user_metadata_layer_0): Linear(in_features=23, out_features=8, bias=True)
  (combined_layer_0): Linear(in_features=76, out_features=16, bias=True)
  (combined_layer_1): Linear(in_features=16, out_features=1, bias=True)
)

While our model works and the results look great, it's not always possible to be able to fully train two separate models like we've done in this tutorial. Sometimes, it's easier (and even better) to train a single hybird model up from scratch, no pretrained ``MatrixFactorizationModel`` needed.

In the next tutorial, we'll cover multi-stage models in Collie, tackling this exact problem and more! See you there! 

----- 